<a href="https://colab.research.google.com/github/anankan10/frontend-tamgramcheck/blob/main/grammar_checking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#command to delete folders which created by simpletranformers training
!rm -R outputs

rm: cannot remove 'outputs': No such file or directory


In [ ]:
!zip -r /content/outputs.zip /content/outputs

In [ ]:
!pip install -q pyngrok

!pip install -q streamlit

!pip install -q streamlit_ace

In [ ]:
#install required packages
#to run the finetuning you need to uncomment below line and run this cell

#!pip install tensorboardx
!pip install simpletransformers

In [ ]:
#import basic packages
import os
import sklearn
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from google.colab import files
import logging
from statistics import mean
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
import re
import nltk

In [ ]:
#import simpletransformers required packages 
from simpletransformers.classification import ClassificationArgs,ClassificationModel


In [ ]:
# Upload the train and test file from your local drive
uploaded = files.upload()

Saving train.tsv to train.tsv
Saving test.tsv to test.tsv


In [ ]:
#read uploaded files and save it as dataframe
df_train = pd.read_csv("train.tsv", 
                       delimiter='\t', 
                       header=None, 
                       names=['text', 'label'])
df_test = pd.read_csv("test.tsv", 
                      delimiter='\t', 
                      header=None, 
                      names=['text', 'label'])

In [ ]:
df_train

,text,label
0,11 பெப்ரவரி அன்று முபாரக் முனைப்புடன் செயல்பட்...,gender
1,மேலே சொன்ன புதைபொருள்‌ ஆராய்ச்சியால்‌ இந்த உண்...,gender
2,கவிமணி தமிழிசையில்‌ தணியாத பற்றுடையவள்.,gender
3,கவிமணி தமிழிசையில்‌ தணியாத பற்றுடையது.,gender
4,வீடு ஒன்றில் 3 மோட்டார் சைக்கிளில் வந்த ஆறுபே...,gender
...,...,...
5057,அவர்கள் போனீர்கள்.,person
5058,அவை போனாய்.,person
5059,அவை போனேன்.,person
5060,அவை போனோம்.,person


In [ ]:
#check test dataframes and their entries unique labels
df_test['label'].value_counts()


number         121
grammatical    120
person         120
gender          80
Name: label, dtype: int64

In [ ]:
#check train dataframes and their entries unique labelsd
df_train['label'].value_counts()

grammatical    2575
person         1002
number          881
gender          557
person           31
number           11
gender            5
Name: label, dtype: int64

In [ ]:
train

,text,label
0,11 பெப்ரவரி அன்று முபாரக் முனைப்புடன் செயல்பட்...,1
1,மேலே சொன்ன புதைபொருள்‌ ஆராய்ச்சியால்‌ இந்த உண்...,1
2,கவிமணி தமிழிசையில்‌ தணியாத பற்றுடையவள்.,1
3,கவிமணி தமிழிசையில்‌ தணியாத பற்றுடையது.,1
4,வீடு ஒன்றில் 3 மோட்டார் சைக்கிளில் வந்த ஆறுபே...,1
...,...,...
5100,அவர்கள் போனீர்கள்.,2
5101,அவை போனாய்.,2
5102,அவை போனேன்.,2
5103,அவை போனோம்.,2


In [ ]:
#convert label to numbers 
def map_label_to_number(df):
  
  # df_test['label'].str.strip()
  label_dict_error_types = {
      'grammatical':0,
      'gender': 1,
      'person': 2,
      'number': 3,
      'gender ':1,
      'number ':3, 
      'person ':2,
      ' gender':1,}
 

  df['label'].str.strip()
  df.label = df.label.replace(label_dict_error_types)
  return df


In [ ]:
# in training set map labels to number and check label counts
train = map_label_to_number(df_train)
train['label'].value_counts()

0    2575
2    1033
3     892
1     562
Name: label, dtype: int64

In [ ]:
# in test set map labels to number and check label counts
test = map_label_to_number(df_test)
test['label'].value_counts()

3    121
2    120
0    120
1     80
Name: label, dtype: int64

In [ ]:
#copy the train, test data and check no of examples
training_data, test_data = train.copy(), test.copy()
print(f"No. of training examples: {training_data.shape[0]}")
print(f"No. of testing examples: {test_data.shape[0]}")

No. of training examples: 5062
No. of testing examples: 441


# Data preprocessing

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def clean_text2(text):
    t1 = re.sub(r"[!@#$%^&*()-_=+{}\|:;?/.>,<''`~]", " ", text)
    t1 = t1.replace('"', ' ')
    words = nltk.word_tokenize(t1)
    wordList = list(words)
    return wordList


In [ ]:
print(clean_text2("எல்லாரும் மரம் வெட்டுவற்காகத் தங்கள் கையில் எடுத்து வந்திருந்த கோடாரியை அப்படியே போட்டுவிட்டுத் தொடர்ந்து சென்றேன்"))

['எல்லாரும்', 'மரம்', 'வெட்டுவற்காகத்', 'தங்கள்', 'கையில்', 'எடுத்து', 'வந்திருந்த', 'கோடாரியை', 'அப்படியே', 'போட்டுவிட்டுத்', 'தொடர்ந்து', 'சென்றேன்']


In [ ]:
# importing pandas as pd
import pandas as pd
  
# dictionary of lists
dict = {'name':["aparna", "pankaj", "sudhir", "Geeku"],
        'degree': ["MBA", "BCA", "M.Tech", "MBA"],
        'score':[90, 40, 80, 98]}
 
# creating a dataframe from a dictionary
df = pd.DataFrame(dict)
 
print(df)

# iterating over rows using iterrows() function
for i, j in df.iterrows():
    print(j['name'])
    print()

In [ ]:
concat_df = pd.concat([training_data,test_data])
word_count = 0
for i,j in concat_df.iterrows():
  word_list = clean_text2(j['text'])
  word_count += len(word_list)
print(word_count)
print(concat_df.shape)
print(f"Average No. of words in the sentence: {word_count/concat_df.shape[0]}")

29423
(5503, 2)
Average No. of words in the sentence: 5.34671997092495


In [ ]:
# define model arguments, we use different hyperparameters to change 
#in here we use optimum hyperparameter values in best results

model_args = {'manual_seed':4,
              'reprocess_input_data': True,
              'overwrite_output_dir': True,
              'wandb_project': 'baseline_revised', # to visualize the training and evaluation results we use wandb
              "wandb_kwargs": {'entity': 'hacko97'} # your user name for wandb
              }

              # model_args = {'learning_rate':3e-5,
              # 'num_train_epochs': 4,
              # "train_batch_size": 16,
              # 'manual_seed':4,
              # 'max_seq_length':128,
              # 'reprocess_input_data': True,
              # 'overwrite_output_dir': True,
              # 'wandb_project': 'baseline_new', # to visualize the training and evaluation results we use wandb
              # "wandb_kwargs": {'entity': 'hacko97'} # your user name for wandb
              # }

In [ ]:
# Create a ClassificationModel using indic-bert-base
#it download models with multi-class classifier layer
#to run this cell you need to install simpletransformers using !pip install simpletransformers
# model = ClassificationModel('albert',
#                             'ai4bharat/indic-bert', 
#                             num_labels = 4, 
#                             args = model_args)

#uncomment appropriate line when using different models
#model = ClassificationModel('bert', 'bert-base-multilingual-cased', num_labels=4, args=args12)
model = ClassificationModel('xlmroberta',
                            'xlm-roberta-base',
                            num_labels=4,
                            args=model_args)
#model = ClassificationModel('electra', 'monsoon-nlp/tamillion', num_labels=4, args=args17)

Downloading:   0%|          | 0.00/512 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:460: UserWarning: use_multiprocessing automatically disabled as xlmroberta fails when using multiprocessing for feature conversion.
  f"use_multiprocessing automatically disabled as {model_type}"


In [ ]:
# Train the model
model.train_model(training_data)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Running Epoch 0 of 1:   0%|          | 0/639 [00:00<?, ?it/s]

(639, 1.251046122519623)

In [ ]:
def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
def print_test_results():
  result, model_outputs, wrong_predictions = model.eval_model(test_data, f1=f1_multiclass, acc=accuracy_score)
  return result
print_test_results()

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/441 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/56 [00:00<?, ?it/s]

Training loss,▅▂▇█▄▁▂▆▃▅▃▆
global_step,▁▂▂▃▄▄▅▅▆▇▇█
lr,█▇▇▆▅▅▄▄▃▂▂▁
Training loss,1.42944
global_step,600
lr,0.0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


{'mcc': 0.0,
 'f1': 0.272108843537415,
 'acc': 0.272108843537415,
 'eval_loss': 1.4867815290178572}

In [ ]:
def predict(sentence):
  predictions, raw_outputs = model.predict([sentence])
  if predictions[0]==0:
    print("grammatical")
  elif predictions[0]==1:
    print("gender")
  elif predictions[0]==2:
    print("person")
  elif predictions[0]==3:
    print("number")
predict(predictions)

In [ ]:
sentence = 'வதனி பாடினான்.'
predict(sentence)


# Deploy grammar checker


In [ ]:
!pip install -q pyngrok

!pip install -q streamlit

!pip install -q streamlit_ace

In [ ]:
%%writefile app.py
import pickle
import streamlit as st
 
# loading the trained model
pickle_in = open('classifier.pkl', 'rb') 
classifier = pickle.load(pickle_in)
@st.cache()
# defining the function which will make the prediction using the data which the user inputs 
def predict(sentence):
    predictions, raw_outputs =classifier.predict([sentence])
    if predictions[0]==0:
        return "grammatical"
    elif predictions[0]==1:
        return "gender"
    elif predictions[0]==2:
        return "person"
    elif predictions[0]==3:
        return "number"

# this is the main function in which we define our webpage  
def main():       
    # front end elements of the web page 
    html_temp = """ 
    <div style ="background-color:yellow;padding:13px"> 
    <h1 style ="color:black;text-align:center;">Streamlit Loan Prediction ML App</h1> 
    </div> 
    """
      
    # display the front end aspect
    st.markdown(html_temp, unsafe_allow_html = True) 
      
    # following lines create boxes in which user can enter data required to make prediction 
    Sentence ='வதனி பாடினான்.'
    Gender = st.selectbox('Gender',("Male","Female"))
    Married = st.selectbox('Marital Status',("Unmarried","Married")) 
    ApplicantIncome = st.number_input("Applicants monthly income") 
    LoanAmount = st.number_input("Total loan amount")
    Credit_History = st.selectbox('Credit_History',("Unclear Debts","No Unclear Debts"))
    result =""
      
    # when 'Predict' is clicked, make the prediction and store it 
    if st.button("Predict"): 
        result = predict(sentence) 
        st.success('Your loan is {}'.format(result))
        print(LoanAmount)
     
    
if __name__=='__main__': 
    main()       


Overwriting app.py


In [ ]:
!streamlit run app.py &>/dev/null&

In [ ]:

from pyngrok import ngrok
 
public_url = ngrok.connect('8501')
public_url

<NgrokTunnel: "http://3a70-35-221-193-229.ngrok.io" -> "http://localhost:8501">

In [ ]:
#it is used for randomly split dataset into train and testing using skilearn
#uncomment this to check against random test
#training_data, test_data = train_test_split(df, test_size=0.1, random_state=25,stratify=df[['label']])
# eval_data, test_data = train_test_split(testing_data, test_size=0.5, random_state=25)
#print(f"No. of training examples: {training_data.shape[0]}")
#print(f"No. of testing examples: {test_data.shape[0]}")
# print(f"No. of testing examples: {test_data.shape[0]}")

In [ ]:
# get names of indexes for which
# column Age has value 21
# index_names = df[ df['label'] == 'tense' ].index
  
# # drop these row indexes
# # from dataFrame
# df.drop(index_names, inplace = True)

# Hyper parameter optimization using Wandb


In [ ]:
import wandb
wandb.login()

# setup wandb environment variables
%env WANDB_ENTITY=hacko97
%env WANDB_PROJECT=ssss

wandb: Currently logged in as: hacko97 (use `wandb login --relogin` to force relogin)


env: WANDB_ENTITY=hacko97
env: WANDB_PROJECT=ssss


In [ ]:
import numpy as np
sweep_config = {
    "name": "vanilla-sweep-batch-16",
    "method": "bayes",
    "metric": {"name": "train_loss", "goal": "minimize"},
    "parameters": {
        "num_train_epochs": {"values":[2,3,4]},
        "train_batch_size": {"values":[16,32]},
        "learning_rate": {"values":[1e-5,2e-5,3e-5,4e-5,5e-5]},

    },
    "early_terminate": {"type": "hyperband", "min_iter": 6,},
}
sweep_id = wandb.sweep(sweep_config, project="ssss")

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model_args = ClassificationArgs()
model_args.eval_batch_size = 32
model_args.evaluate_during_training = True
model_args.evaluate_during_training_silent = False
model_args.evaluate_during_training_steps = -1
model_args.save_eval_checkpoints = False
model_args.save_model_every_epoch = False
model_args.learning_rate = 1e-5
model_args.manual_seed = 4
model_args.max_seq_length = 256
model_args.multiprocessing_chunksize = 5000
model_args.no_cache = True
model_args.num_train_epochs = 3
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.train_batch_size = 16
model_args.gradient_accumulation_steps = 2

model_args.output_dir = "default_output"
model_args.best_model_dir = "default_output/best_model"
model_args.wandb_project = "ssss"
model_args.wandb_kwargs = {"entity": "hacko97"}

from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
def train():
    # Initialize a new wandb run
    wandb.init()

    # Create a TransformerModel
    model = ClassificationModel('albert', 'ai4bharat/indic-bert', num_labels=4, use_cuda=True, cuda_device=0, args=model_args)


    # Train the model
    model.train_model(
        training_data,
        eval_df=test_data,
      
    )

    # Sync wandb
    wandb.join()

wandb.agent(sweep_id, train)

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


In [ ]:
train_args={
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'learning_rate':3e-5,
      'num_train_epochs': 4,
      "train_batch_size": 16,
      'manual_seed':4,
     
    'wandb_project': 'baseline',
    "wandb_kwargs": {'entity': 'hacko97'}
}
model = ClassificationModel('albert', 'ai4bharat/indic-bert', num_labels=4, use_cuda=True, args=train_args)


Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForSequenceClassification: ['predictions.dense.weight', 'sop_classifier.classifier.bias', 'predictions.decoder.bias', 'sop_classifier.classifier.weight', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.bias', 'predictions.decoder.weight', 'predictions.dense.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indi

In [ ]:
model.train_model(
    training_data
)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/2052 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_albert_128_4_2


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Initializing WandB run for training.


Running Epoch 0 of 4:   0%|          | 0/129 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/129 [00:00<?, ?it/s]

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test_data, acc=sklearn.metrics.accuracy_score)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/228 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_albert_128_4_2


Running Evaluation:   0%|          | 0/29 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Initializing WandB run for evaluation.


Training loss,▆▇▄█▆▅▁▅▃▇
global_step,▁▂▃▃▄▅▆▆▇█
lr,█▇▆▆▅▄▃▃▂▁
Training loss,1.35138
global_step,500
lr,0.0


INFO:simpletransformers.classification.classification_model:{'mcc': 0.23547272863670207, 'acc': 0.49122807017543857, 'eval_loss': 1.1373380463698815}


In [ ]:
# model_args = ClassificationArgs()
# model_args.learning_rate = 3e-5
# model_args.num_train_epochs = 4
# model_args.train_batch_size = 16
# model_args.manual_seed = 4
# model_args.reprocess_input_data = True
# model_args.overwrite_output_dir = True


##Simple training

In [ ]:
args1= {'learning_rate':1e-5,
       'num_train_epochs': 2,
       "train_batch_size": 16,
        'manual_seed':4,
       'reprocess_input_data': True,
    'overwrite_output_dir': True,
     'wandb_project': 'baseline_xlm_r',
    "wandb_kwargs": {'entity': 'hacko97'}
       }

args2= {'learning_rate':1e-5,
       'num_train_epochs': 2,
       "train_batch_size": 32,
 'manual_seed':4,
       'reprocess_input_data': True,
    'overwrite_output_dir': True,
     'wandb_project': 'baseline_xlm_r',
    "wandb_kwargs": {'entity': 'hacko97'}
       }

args3= {'learning_rate':1e-5,
       'num_train_epochs': 3,
       "train_batch_size": 16,
 'manual_seed':4,
       'reprocess_input_data': True,
    'overwrite_output_dir': True,
     'wandb_project': 'baseline',
    "wandb_kwargs": {'entity': 'hacko97'}
       }
args4= {'learning_rate':1e-5,
       'num_train_epochs': 3,
       "train_batch_size": 32,
 'manual_seed':4,
       'reprocess_input_data': True,
    'overwrite_output_dir': True,
     'wandb_project': 'baseline_xlm_r',
    "wandb_kwargs": {'entity': 'hacko97'}
       }
args5= {'learning_rate':1e-5,
       'num_train_epochs': 4,
       "train_batch_size": 16,
 'manual_seed':4,
       'reprocess_input_data': True,
    'overwrite_output_dir': True,
     'wandb_project': 'baseline_xlm_r',
    "wandb_kwargs": {'entity': 'hacko97'}
       }
args6= {'learning_rate':1e-5,
       'num_train_epochs': 4,
       "train_batch_size": 32,
 'manual_seed':4,
       'reprocess_input_data': True,
    'overwrite_output_dir': True,
     'wandb_project': 'baseline_xlm_r',
    "wandb_kwargs": {'entity': 'hacko97'}
       }
       
  

In [ ]:
args7= {'learning_rate':2e-5,
       'num_train_epochs': 2,
       "train_batch_size": 16,
        'manual_seed':4,
       'reprocess_input_data': True,
    'overwrite_output_dir': True,
     'wandb_project': 'baseline_xlm_r',
    "wandb_kwargs": {'entity': 'hacko97'}
       }

args8= {'learning_rate':2e-5,
       'num_train_epochs': 2,
       "train_batch_size": 32,
 'manual_seed':4,
       'reprocess_input_data': True,
    'overwrite_output_dir': True,
     'wandb_project': 'baseline_xlm_r',
    "wandb_kwargs": {'entity': 'hacko97'}
       }

args9= {'learning_rate':2e-5,
      'num_train_epochs': 3,
       "train_batch_size": 16,
 'manual_seed':4,
       'reprocess_input_data': True,
    'overwrite_output_dir': True,
     'wandb_project': 'baseline_xlm_r',
    "wandb_kwargs": {'entity': 'hacko97'}
       }
args10= {'learning_rate':2e-5,
       'num_train_epochs': 3,
       "train_batch_size": 32,
 'manual_seed':4,
       'reprocess_input_data': True,
    'overwrite_output_dir': True,
     'wandb_project': 'baseline_xlm_r',
    "wandb_kwargs": {'entity': 'hacko97'}
       }
args11= {'learning_rate':2e-5,
       'num_train_epochs': 4,
       "train_batch_size": 16,
 'manual_seed':4,
       'reprocess_input_data': True,
    'overwrite_output_dir': True,
     'wandb_project': 'baseline_xlm_r',
    "wandb_kwargs": {'entity': 'hacko97'}
       }
args12= {'learning_rate':2e-5,
       'num_train_epochs': 4,
       "train_batch_size": 32,
 'manual_seed':4,
       'reprocess_input_data': True,
    'overwrite_output_dir': True,
     'wandb_project': 'baseline_xlm_r',
    "wandb_kwargs": {'entity': 'hacko97'}
       }
       

In [ ]:
args13= {'learning_rate':3e-5,
       'num_train_epochs': 2,
       "train_batch_size": 16,
        'manual_seed':4,
       'reprocess_input_data': True,
    'overwrite_output_dir': True,
     'wandb_project': 'baseline_xlm_r',
    "wandb_kwargs": {'entity': 'hacko97'}
       }

args14= {'learning_rate':3e-5,
       'num_train_epochs': 2,
       "train_batch_size": 32,
 'manual_seed':4,
       'reprocess_input_data': True,
    'overwrite_output_dir': True,
     'wandb_project': 'baseline_xlm_r',
    "wandb_kwargs": {'entity': 'hacko97'}
       }

args15= {'learning_rate':3e-5,
      'num_train_epochs': 3,
       "train_batch_size": 16,
 'manual_seed':4,
       'reprocess_input_data': True,
    'overwrite_output_dir': True,
     'wandb_project': 'baseline_xlm_r',
    "wandb_kwargs": {'entity': 'hacko97'}
       }
args16= {'learning_rate':3e-5,
       'num_train_epochs': 3,
       "train_batch_size": 32,
 'manual_seed':4,
       'reprocess_input_data': True,
    'overwrite_output_dir': True,
     'wandb_project': 'baseline_xlm_r',
    "wandb_kwargs": {'entity': 'hacko97'}
       }
args17= {'learning_rate':3e-5,
       'num_train_epochs': 4,
       "train_batch_size": 16,
 'manual_seed':4,
 'max_seq_length':256,
       'reprocess_input_data': True,
    'overwrite_output_dir': True,
     'wandb_project': 'baseline_xlm_r',
    "wandb_kwargs": {'entity': 'hacko97'}
       }
args18= {'learning_rate':3e-5,
       'num_train_epochs': 4,
       "train_batch_size": 32,
 'manual_seed':4,
       'reprocess_input_data': True,
    'overwrite_output_dir': True,
     'wandb_project': 'baseline_xlm_r',
    "wandb_kwargs": {'entity': 'hacko97'}
       }
       

In [ ]:
args19= {'learning_rate':4e-5,
       'num_train_epochs': 2,
       "train_batch_size": 16,
        'manual_seed':4,
       'reprocess_input_data': True,
    'overwrite_output_dir': True,
     'wandb_project': 'baseline_xlm_r',
    "wandb_kwargs": {'entity': 'hacko97'}
       }

args20= {'learning_rate':4e-5,
       'num_train_epochs': 2,
       "train_batch_size": 32,
 'manual_seed':4,
       'reprocess_input_data': True,
    'overwrite_output_dir': True,
     'wandb_project': 'baseline_xlm_r',
    "wandb_kwargs": {'entity': 'hacko97'}
       }

args21= {'learning_rate':4e-5,
      'num_train_epochs': 3,
       "train_batch_size": 16,
 'manual_seed':4,
       'reprocess_input_data': True,
    'overwrite_output_dir': True,
     'wandb_project': 'baseline',
    "wandb_kwargs": {'entity': 'hacko97'}
       }
args22= {'learning_rate':2e-5,
       'num_train_epochs': 3,
       "train_batch_size": 32,
 'manual_seed':4,
       'reprocess_input_data': True,
    'overwrite_output_dir': True,
     'wandb_project': 'baseline',
    "wandb_kwargs": {'entity': 'hacko97'}
       }
args23= {'learning_rate':4e-5,
       'num_train_epochs': 4,
       "train_batch_size": 16,
 'manual_seed':4,
       'reprocess_input_data': True,
    'overwrite_output_dir': True,
     'wandb_project': 'baseline',
    "wandb_kwargs": {'entity': 'hacko97'}
       }
args24= {'learning_rate':4e-5,
       'num_train_epochs': 4,
       "train_batch_size": 32,
 'manual_seed':4,
       'reprocess_input_data': True,
    'overwrite_output_dir': True,
     'wandb_project': 'baseline',
    "wandb_kwargs": {'entity': 'hacko97'}
       }
       

In [ ]:
args29= {'learning_rate':5e-5,
       'num_train_epochs': 4,
       "train_batch_size": 16,
 'manual_seed':4,
       'reprocess_input_data': True,
    'overwrite_output_dir': True,
     'wandb_project': 'baseline',
    "wandb_kwargs": {'entity': 'hacko97'}}

In [ ]:
args30= {'learning_rate':5e-5,
       'num_train_epochs': 4,
       "train_batch_size": 32,
       
 'manual_seed':4,
       'reprocess_input_data': True,
    'overwrite_output_dir': True,
     'wandb_project': 'baseline',
    "wandb_kwargs": {'entity': 'hacko97'}
       }

In [ ]:
# from simpletransformers.classification import ClassificationModel

	
# Create a ClassificationModel
#model = ClassificationModel('albert', 'ai4bharat/indic-bert', num_labels=4, args=args17)
#model = ClassificationModel('bert', 'bert-base-multilingual-cased', num_labels=4, args=args12)
model = ClassificationModel('xlmroberta', 'xlm-roberta-base', num_labels=4, args=args17)
#model = ClassificationModel('electra', 'monsoon-nlp/tamillion', num_labels=4, args=args17)

Downloading:   0%|          | 0.00/512 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:460: UserWarning: use_multiprocessing automatically disabled as xlmroberta fails when using multiprocessing for feature conversion.
  f"use_multiprocessing automatically disabled as {model_type}"


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
# Train the model
model.train_model(training_data)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Running Epoch 0 of 4:   0%|          | 0/327 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/327 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/327 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/327 [00:00<?, ?it/s]

(1308, 0.9174061428516282)

In [ ]:
#result, model_outputs, wrong_predictions = model.eval_model(test_data)

In [ ]:
#result, model_outputs

In [ ]:
test_data

,text,label
0,11 பெப்ரவரி அன்று முபாரக் முனைப்புடன் செயல்பட்...,0
1,மேலே சொன்ன புதைபொருள்‌ ஆராய்ச்சியால்‌ இந்த உண்...,0
2,கவிமணி தமிழிசையில்‌ தணியாத பற்றுடையவர்‌.,0
3,கவிமணியின்‌ பாடல்கள்‌ எளிமையும்‌ இனிமையும்‌ நி...,0
4,வீடு ஒன்றில் 3 மோட்டார் சைக்கிளில் வந்த ஆறுபே...,0
...,...,...
465,வழிபடு தெய்வமான கண்ணனைச்‌ சேவகனாக்கியது. தமிழ்...,3
466,கவிதைப்‌ பெண்ணை விடுதலை செய்தவள் பாரதி.,3
467,அந்தப்‌ பெண்ணை ஆணாக்கி அவள்‌ கையில்‌ ஆயுதமும்‌...,3
468,பழம்‌ புலவர்களின்‌ விருத்தங்களைவிட மக்கள்‌ பாட...,3


In [ ]:
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result, model_outputs, wrong_predictions = model.eval_model(test_data, f1=f1_multiclass, acc=accuracy_score)

In [ ]:
result
predictions, raw_outputs = model.predict(['வதனி பாடினான்.'])
raw_outputs
def predict(predictions):
  if predictions[0]==0:
    print("grammatical")
  elif predictions[0]==1:
    print("gender")
  elif predictions[0]==2:
    print("person")
  elif predictions[0]==3:
    print("number")
predict(predictions)

{'mcc': 0.22357227321006726,
 'f1': 0.37872340425531914,
 'acc': 0.37872340425531914,
 'eval_loss': 1.5831491502664856}

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array([[-0.11621094,  0.18212891,  0.00515747, -0.19580078]])

gender


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
import logging

import pandas as pd
import sklearn

import wandb
from simpletransformers.classification import (
    ClassificationArgs,
    ClassificationModel,
)

sweep_config = {
    "method": "bayes",  # grid, random
    "metric": {"name": "train_loss", "goal": "minimize"},
    "parameters": {
        "num_train_epochs": {"values": [2, 3, 5]},
        "learning_rate": {"min": 5e-5, "max": 4e-4},
    },
}

sweep_id = wandb.sweep(sweep_config, project="Simple Sweep")

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Preparing train data
train_data = [
    ["Aragorn was the heir of Isildur", "True"],
    ["Frodo was the heir of Isildur", "False"],
]
train_df = pd.DataFrame(train_data)
train_df.columns = ["text", "labels"]

# Preparing eval data
eval_data = [
    ["Theoden was the king of Rohan", "True"],
    ["Merry was the king of Rohan","False"],
]
eval_df = pd.DataFrame(eval_data)
eval_df.columns = ["text", "labels"]

model_args = ClassificationArgs()
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.evaluate_during_training = True
model_args.manual_seed = 4
model_args.use_multiprocessing = True
model_args.train_batch_size = 16
model_args.eval_batch_size = 8
model_args.labels_list = ["True", "False"]
model_args.wandb_project = "Simple Sweep"

def train():
    # Initialize a new wandb run
    wandb.init()

    # Create a TransformerModel
    model = ClassificationModel(
        "roberta",
        "roberta-base",
        use_cuda=True,
        args=model_args,
        sweep_config=wandb.config,
    )

    # Train the model
    model.train_model(train_df, eval_df=eval_df)

    # Evaluate the model
    model.eval_model(eval_df)

    # Sync wandb
    wandb.join()


wandb.agent(sweep_id, train)